### Finding the best-fit radiation structure -- Running Emis3D

In this example, we finally run the program. Prior to this we need to make sure that the following steps have already been completed:
1. Correct implimentation of the tokamak within the Emis3D enviroment. 
2. Experimental data is formatted and saved in the directory listed within the run config file. 
3. Many artifical radiation distributions (radDists) are created and the directories are listed in the run config file. 

---

#### Correct implimentation of the tokamak within the Emis3D enviroment

This is the first thing that should be done when adding a new tokamak to Emis3D. You'll need files such as the tokamak configuration file (example: `tokamaks/DIII-D/DIII-D_settings.yaml`) and information about each bolometer. There are additional optional files such as a 3D CAD file of the outer wall, which is used to build the observable universe. See the `sxrChordPlotter.ipynb` demo for more information about bolometer implementation.

#### Experimental data

The experimental data should be calibrated and processed prior to running the main Emis3D program. The older method to incorporate everything into one large package was removed during the refactor since grabbing and calibrating data is unique to every machine. We have included several programs on how to do this with DIII-D under `tokamaks/DIII-D/sxrSavers/`. The important thing to note is that it is required to have these variables in the h5 experimental data file: ["channelOrder","TIME","DATA_CALIBRATED","DATA_CALIBRATED_ERROR","DATA_CALIBRATED_UNITS"]. The program will then compare the experimental channel order to that done when creating radDists and it is important that the names match in the bolometer configuration file (which is used when creating radDists) and the saved data. 

The general process that is used for DIII-D is to:
1. Grab, filter, correct for offset, calibrate the data, then save it under `inputs/DIII-D/sxrData/{shot number}`
2. Cross-calibrate the data during the current quench (since DIII-D's bolometers are not absolutly calibrated) by using `crossCalibrateSXR.py`, put those scale factors in the run config file:


>BOLOMETERS:  
> SX90PF:  
>> dataFileName: '184407/184407_SX90PF_PROCESSED.h5'   
>> scalingFactor: 5.00 # Multiply the calibrated data by this factor  
>  
> SX90MF:  
>> dataFileName: '184407/184407_SX90MF_PROCESSED.h5'   
>> scalingFactor: 4.96    
>     
> DISRADU:     
>> dataFileName: '184407/184407_DISRADU_PROCESSED.h5'    
>> scalingFactor: 1.0    

#### Many artifical mock radiation signals

The code basically minimizes the previously created mock radiation signals multiplied by some factor to the experimental data. Therefore, the better (or more realistic) radDists we have, the better the fit with data. Typically, for SPI discharges, helical distributions fit best during the initial stages, followed by elongated rings for the remaining of the discharge. See the `radDist` example for more information here. 

---

### Running Emis3D

Now that all of the prerequisites are in order, we can finally run the program. An example script can be found under `runProgramScript/run_emis3D.py`. But we will have a non-parallelized version below. 

In [ ]:
import os
import sys

# Ensure the parent directory is in the system path for module imports
sys.path.append(os.path.abspath("../.."))
import time
import numpy as np
import main.Emis3D as Emis3D
import main.Util_emis3D as Util_emis3D

Now we need to modify the input parameters and tell the program where the run configuration is located

In [ ]:
# --- Update these parameters:
evalTimes = [2124.0]  # np.arange(2119, 2127, 0.3)
tokamakName = "DIII-D"
runConfigName = "184407/184407_runConfig.yaml"
verbose = True